In [2]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import time
import re

In [38]:
catalog = requests.get('http://www.starcitygames.com/catalog/magic_the_gathering/').content
catalog_web = BeautifulSoup(catalog, 'html')
links_to_categories = catalog_web.select('ul.cardset_lists a[href^= "http://www.starcitygames.com/catalog/category/"]')
links = [link['href'] for link in links_to_categories]
links

['http://www.starcitygames.com/catalog/category/10th Edition',
 'http://www.starcitygames.com/catalog/category/3rd EditionRevised',
 'http://www.starcitygames.com/catalog/category/4th Edition',
 'http://www.starcitygames.com/catalog/category/5th Edition',
 'http://www.starcitygames.com/catalog/category/6th Edition',
 'http://www.starcitygames.com/catalog/category/7th Edition',
 'http://www.starcitygames.com/catalog/category/8th Edition',
 'http://www.starcitygames.com/catalog/category/9th Edition',
 'http://www.starcitygames.com/catalog/category/Aether Revolt',
 'http://www.starcitygames.com/catalog/category/Alara Reborn',
 'http://www.starcitygames.com/catalog/category/Alliances',
 'http://www.starcitygames.com/catalog/category/Alpha',
 'http://www.starcitygames.com/catalog/category/Alternate 4th Edition',
 'http://www.starcitygames.com/catalog/category/Amonkhet',
 'http://www.starcitygames.com/catalog/category/Anthologies',
 'http://www.starcitygames.com/catalog/category/Antiquities'

In [40]:
def card_parser(request):
    catalog = request
    table = pd.read_html(catalog)
    df_table = table[1]

    df_table = df_table.drop(0)
    name = df_table.iloc[0,]
    df_table = df_table.rename(name, axis = 1)
    df_table = df_table.drop(1)
    df_table = df_table.drop(['Mana', 'Stock'], axis = 1)

    #df_table.columns[7]
    df_table = df_table[df_table.columns[:-1]]
    df_table = df_table.dropna(axis = 0, subset = ['Name', 'Category'])
    df_table = df_table.reset_index(drop = True)

    for doble_price in df_table.Price:
        fix_price = re.findall('..\...$', doble_price)
        df_table.Price = df_table.Price.replace({doble_price: fix_price})
    return df_table

In [37]:
def next_parser(request):   
    try:
        card_page = request
        card_page = BeautifulSoup(card_page, 'html')
        next_link = card_page.select('section.box_sizing_fix#content div[style] a:contains(Next)')
        next_link = next_link[0]['href']
        return next_link
    except:
        return None

In [48]:
links = ['http://www.starcitygames.com/catalog/category/10th Edition',
 'http://www.starcitygames.com/catalog/category/3rd EditionRevised']

df_papa = []

for url in links: # itero sobre la lista de vínculos de las expansiones 
    df_expansion = [] # lista vacía donde voy a colocar el df
    request = requests.get(url).content # hago el request de la página y lo guardo en una variable 
    next_link = next_parser(request) # obtengo con una función el link a la siguiente página
    df_expansion.append(card_parser(request)) # corre la función que extrae el df con la tabla limpia y 
                                              # después lo coloca en la lista df_expansion. Esto por si la expansión tiene
                                              # una sola página
    while next_link: # mientras haya un vínculo a la próxima página haz lo siguiente
        request = requests.get(next_link).content  # vuelve a hacer la peticion 
        next_link = next_parser(request)           # recoge el link a la siguiente página
        df_expansion.append(card_parser(request))  # toma el df limpio 
    if len(df_expansion) > 0:
        df_expansion = pd.concat(df_expansion, ignore_index = True)
    df_papa.append(df_expansion) # coloca todos los dataframe extraidos en una lista de que agrupa todos los data frames

In [55]:
df_papa[1]

,Name,Category,Type,P/T,Rarity,Condition,Price
0,Air Elemental,3rd Edition/Revised,Creature - Elemental,4/4,U,NM/M,$0.29
1,Aladdin's Lamp,3rd Edition/Revised,Artifact,NaN,R,NM/M,$0.99
2,Aladdin's Ring,3rd Edition/Revised,Artifact,NaN,R,NM/M,$0.59
3,Animate Artifact,3rd Edition/Revised,Enchantment - Aura,NaN,U,NM/M,$0.25
4,Animate Dead,3rd Edition/Revised,Enchantment - Aura,NaN,U,NM/M,$3.99
5,Animate Wall,3rd Edition/Revised,Enchantment - Aura,NaN,R,NM/M,$0.49
6,Ankh of Mishra,3rd Edition/Revised,Artifact,NaN,R,NM/M,$3.99
7,Armageddon,3rd Edition/Revised,Sorcery,NaN,R,NM/M,$7.99
8,Armageddon Clock,3rd Edition/Revised,Artifact,NaN,R,NM/M,$0.59
9,Aspect of Wolf,3rd Edition/Revised,Enchantment - Aura,NaN,R,NM/M,$0.99


In [ ]:
a = 'http://www.starcitygames.com/catalog/category/10th Edition'
b = re.findall(r'\/.+$)